In [2]:
from fastai.vision.all import *
from random import randint
from ml_utils import *
import os

In [3]:
pd.set_option('display.max_rows', None)

In [4]:
clips_df = pd.read_csv('data/labels.csv')

# dropping dangling columns
clips_df = clips_df.loc[:, ~clips_df.columns.str.contains('^Unnamed')]

def clean_labels(c): return str([(i).strip().replace(" ", "_").replace("-", "_").replace("/", "_") for i in re.sub(r"[\[\]]",'',c).split(',')]).replace("'","").strip("'").strip("[").strip("]").replace(" ","").replace('monopolar_curved_scissors','monopolar_curved_scissor')
clips_df['tools_present'] = clips_df['tools_present'].apply(clean_labels)
clips_df.head()

,clip_name,tools_present
0,clip_000000,"needle_driver,nan,needle_driver,cadiere_forceps"
1,clip_000001,"needle_driver,nan,needle_driver,cadiere_forceps"
2,clip_000002,"needle_driver,nan,needle_driver,cadiere_forceps"
3,clip_000003,"needle_driver,nan,needle_driver,cadiere_forceps"
4,clip_000004,"needle_driver,nan,needle_driver,cadiere_forceps"


In [5]:
df=clips_df.groupby('tools_present').nth(0)
len(df)

153

In [6]:
src=Path('data/train_images_crop')
dst=Path('data/testing/images')

In [7]:
for i in get_image_files(dst): os.remove(i) 

In [8]:
for clip in df.clip_name:
    fs = get_image_files(src/clip)
    if len(fs)>0:
        fno=randint(0,len(fs)-1)
        im=PILImage.create(fs[fno])
        fn=str(dst)+'/'+clip+'_'+fs[fno].name
        im.save(fn)
    

In [9]:
ensem_path=Path("test/models")

In [13]:
ensem_learner=[load_learner(m, cpu=False) for m in ensem_path.ls() if m.suffix=='.pkl']

In [14]:
fs=get_image_files(dst);len(fs)

153

In [15]:
tta_res=[]
prs_items=[]
for learn in ensem_learner:
    tta_res.append(learn.tta(dl=learn.dls.test_dl(fs)))
    if len(prs_items)<1:
        prs_items=learn.dl.items

tta_prs=first(zip(*tta_res))
tta_prs=torch.stack(tta_prs)

lbls=[]
for i in range(len(c)):
    arm_preds = tta_prs[:,:,cfg(i):cfg(i+1)].mean(0);
    arm_idxs = arm_preds.argmax(dim=1)
    arm_vocab = np.array(vocab[i])
    lbls.append(arm_vocab[arm_idxs])


In [12]:
# data=[]
# for lbl,f in zip(lbls,prs_items):
#     l1,l2,l3,l4=lbls
#     data.append((f,l1+','+l2+','+l3+','+l4))
# df = pd.DataFrame(data,columns=['image_id','PR_labels'])
# df

In [16]:
data=[]
for l1,l2,l3,l4,f in zip(lbls[0],lbls[1],lbls[2],lbls[3],prs_items):
    data.append((f,l1+','+l2+','+l3+','+l4))

In [17]:
clips_df.set_index('clip_name',inplace=True,drop=True)

In [18]:
def get_GT_labels(f):
    c=str(Path(f).name).rpartition('_')[0]
    return clips_df.loc[c,'tools_present']

In [19]:
# df = pd.DataFrame(data,columns=['image_id','PR_labels'])
# df

In [20]:
df = pd.DataFrame(data,columns=['image_id','PR_labels'])
df['GT_labels']=df['image_id'].map(get_GT_labels) 
df['accurate_tf'] = (df['PR_labels']==df['GT_labels'])
#df

In [21]:
print(f"The predictive performance of TeamZERO's model is {round(np.sum(df['accurate_tf'].astype(int))/len(df)*100,2)}%.")

The predictive performance of TeamZERO's model is 98.04%.


In [29]:
l=['bipolar_dissector', 'bipolar_forceps', 'blank'];l

['bipolar_dissector', 'bipolar_forceps', 'blank']

In [32]:
[l for i in range(4)]

[['bipolar_dissector', 'bipolar_forceps', 'blank'],
 ['bipolar_dissector', 'bipolar_forceps', 'blank'],
 ['bipolar_dissector', 'bipolar_forceps', 'blank'],
 ['bipolar_dissector', 'bipolar_forceps', 'blank']]

In [22]:
df

,image_id,PR_labels,GT_labels,accurate_tf
0,data/testing/images/clip_022421_00630.jpg,"force_bipolar,cadiere_forceps,nan,vessel_sealer","force_bipolar,cadiere_forceps,nan,vessel_sealer",True
1,data/testing/images/clip_015431_00585.jpg,"bipolar_forceps,nan,stapler,grasping_retractor","bipolar_forceps,nan,stapler,grasping_retractor",True
2,data/testing/images/clip_013016_00465.jpg,"force_bipolar,nan,cadiere_forceps,stapler","force_bipolar,nan,cadiere_forceps,stapler",True
3,data/testing/images/clip_010093_01125.jpg,"cadiere_forceps,nan,permanent_cautery_hook_spatula,force_bipolar","cadiere_forceps,nan,permanent_cautery_hook_spatula,force_bipolar",True
4,data/testing/images/clip_023092_01260.jpg,"grasping_retractor,bipolar_forceps,nan,stapler","grasping_retractor,bipolar_forceps,nan,stapler",True
5,data/testing/images/clip_010550_00930.jpg,"force_bipolar,bipolar_forceps,nan,permanent_cautery_hook_spatula","force_bipolar,bipolar_forceps,nan,permanent_cautery_hook_spatula",True
6,data/testing/images/clip_000318_01185.jpg,"cadiere_forceps,bipolar_forceps,nan,monopolar_curved_scissor","cadiere_forceps,bipolar_forceps,nan,monopolar_curved_scissor",True
7,data/testing/images/clip_005498_00810.jpg,"cadiere_forceps,monopolar_curved_scissor,nan,force_bipolar","cadiere_forceps,monopolar_curved_scissor,nan,force_bipolar",True
8,data/testing/images/clip_009012_00810.jpg,"prograsp_forceps,prograsp_forceps,nan,permanent_cautery_hook_spatula","prograsp_forceps,prograsp_forceps,nan,permanent_cautery_hook_spatula",True
9,data/testing/images/clip_020548_01680.jpg,"prograsp_forceps,bipolar_forceps,nan,bipolar_forceps","prograsp_forceps,bipolar_forceps,nan,bipolar_forceps",True
